# Gravity data processing

In [ ]:
import numpy as np
import pandas as pd
import xarray as xr
import pyproj
import pygmt

import ensaio
import verde as vd
import boule as bl
import harmonica as hm

## Download gravity data

In [ ]:
gmt_projection = "M12c"

fig = pygmt.Figure()
vmin, vmax =  data.gravity_mgal.min(), data.gravity_mgal.max()
pygmt.makecpt(cmap="viridis", series=[vmin, vmax])
fig.plot(
    x=data.longitude,
    y=data.latitude,
    fill=data.gravity_mgal,
    cmap=True,
    style="c2p",
    projection=gmt_projection,
    frame="af",
)
fig.coast(water="#8fcae7")
fig.colorbar(position="JMR+o0.5c/0c+w8.5c/0.3c", frame=['af', 'y+l"mGal"'])
fig.show()

## Download geoid to convert heights to ellipsoidal heights

In [ ]:
fig = pygmt.Figure()
maxabs = vd.maxabs(geoid)
pygmt.makecpt(series=[-maxabs, maxabs], cmap="polar+h")
fig.grdimage(
    geoid,
    projection="W10c",
    shading="+a45+nt0.2",
)
fig.basemap(frame=["af", "WEsn"])
fig.colorbar(
    position="JCB+w10c",
    frame=["af", 'y+l"m"', 'x+l"geoid height [m]"'],
)
fig.show()

## Cut datasets to the Bushveld Igneous Complex

In [ ]:
region = (25, 32, -28, -23)

In [ ]:
fig = pygmt.Figure()
vmin, vmax =  data.gravity_mgal.min(), data.gravity_mgal.max()
pygmt.makecpt(cmap="viridis", series=[vmin, vmax])
fig.plot(
    x=data.longitude,
    y=data.latitude,
    fill=data.gravity_mgal,
    cmap=True,
    style="c2p",
    projection=gmt_projection,
    frame="af",
)
fig.plot(
    x=[region[0], region[1], region[1], region[0], region[0]],
    y=[region[3], region[3], region[2], region[2], region[3]],
    pen="2p,orangered1,-",
)
fig.coast(water="#8fcae7")
fig.colorbar(position="JMR+o0.5c/0c+w8.5c/0.3c", frame=['af', 'y+l"mGal"'])
fig.show()

In [ ]:
inside = vd.inside((data.longitude, data.latitude), region)
data = data[inside]
data

In [ ]:
fig = pygmt.Figure()
vmin, vmax =  data.gravity_mgal.min(), data.gravity_mgal.max()
pygmt.makecpt(cmap="viridis", series=[vmin, vmax])
fig.plot(
    x=data.longitude,
    y=data.latitude,
    fill=data.gravity_mgal,
    cmap=True,
    style="c3p",
    projection=gmt_projection,
    frame="af",
)
fig.coast(water="#8fcae7")
fig.colorbar(position="JMR+o0.5c/0c+w8.5c/0.3c", frame=['af', 'y+l"mGal"'])
fig.show()

In [ ]:
region_pad = vd.pad_region(region, pad=5)
region_pad

In [ ]:
geoid = geoid.sel(longitude=slice(*region_pad[:2]), latitude=slice(*region_pad[2:]))
geoid

In [ ]:
fig = pygmt.Figure()
maxabs = vd.maxabs(geoid)
pygmt.makecpt(series=[-maxabs, maxabs], cmap="polar+h")
fig.grdimage(
    geoid,
    projection=gmt_projection,
    shading="+a45+nt0.2",
)
fig.basemap(frame=["af", "WEsn"])
fig.colorbar(
    position="JCB+w10c",
    frame=["af", 'y+l"m"', 'x+l"geoid height [m]"'],
)
fig.show()

## Project geographic data and get ellipsoidal heights

In [ ]:
projection = pyproj.Proj(proj="merc", lat_ts=data.latitude.mean())

In [ ]:
easting, northing = projection(data.longitude.values, data.latitude.values)
data = data.assign(easting_m=easting, northing_m=northing)
data

In [ ]:
geoid_proj = vd.project_grid(geoid, projection, method="nearest")
geoid_proj

In [ ]:
# Unravel the grid so that we can pass it to the interpolator
geoid_table = vd.grid_to_table(geoid_proj)
interpolator = vd.Cubic()
interpolator.fit((geoid_table.easting, geoid_table.northing), geoid_table.geoid)

# Predict the geoid height at same locations as the observation points
data = data.assign(geoid_m=interpolator.predict((data.easting_m, data.northing_m)))
data = data.assign(height_geometric_m=data.height_sea_level_m + data.geoid_m)
data

## Compute gravity disturbance

In [ ]:
fig = pygmt.Figure()
maxabs =  vd.maxabs(data.disturbance_mgal)
pygmt.makecpt(cmap="polar+h", series=[-maxabs, maxabs])
fig.plot(
    x=data.longitude,
    y=data.latitude,
    fill=data.disturbance_mgal,
    cmap=True,
    style="c3p",
    projection=gmt_projection,
    frame="af",
)
fig.coast(water="#8fcae7")
fig.colorbar(position="JMR+o0.5c/0c+w8.5c/0.3c", frame=['af', 'y+l"mGal"'])
fig.show()

## Terrain correction

In [ ]:
fname = ensaio.fetch_earth_topography(version=1)
topography = xr.load_dataarray(fname)
topography

In [ ]:
topography = topography.sel(longitude=slice(*region_pad[:2]), latitude=slice(*region_pad[2:]))
topography

In [ ]:
fig = pygmt.Figure()
fig.grdimage(topography, projection=gmt_projection, cmap="etopo1", frame=True)
fig.colorbar(frame='af+l"topography [m]"')
fig.coast(shorelines="white", area_thresh=1e6)

maxabs =  vd.maxabs(data.disturbance_mgal)
pygmt.makecpt(cmap="polar+h", series=[-maxabs, maxabs])
fig.plot(
    x=data.longitude,
    y=data.latitude,
    fill=data.disturbance_mgal,
    cmap=True,
    style="c1p",
    projection=gmt_projection,
)

fig.show()

In [ ]:
topography_proj = vd.project_grid(topography, projection, method="nearest")
topography_proj

In [ ]:
topography_geometric = topography_proj + geoid_proj
topography_geometric

In [ ]:
topography_density = np.where(topography_geometric > 0, 2670, 1040 - 2670)

# Create the prism model of the topographic masses
topography_model = hm.prism_layer(
    coordinates=(topography_geometric.easting, topography_geometric.northing),
    surface=topography_geometric,
    reference=0,
    properties={"density": topography_density}
)
topography_model

In [ ]:
coordinates = (data.easting_m, data.northing_m, data.height_geometric_m)

terrain_effect =  topography_model.prism_layer.gravity(coordinates, field="g_z")
bouguer = data.disturbance_mgal - terrain_effect

data = data.assign(terrain_effect_mgal=terrain_effect)
data = data.assign(gravity_bouguer_mgal=bouguer)
data

In [ ]:
fig = pygmt.Figure()
maxabs = vd.maxabs(data.gravity_bouguer_mgal)
pygmt.makecpt(cmap="polar+h", series=[-maxabs, maxabs])
fig.plot(
    x=data.longitude,
    y=data.latitude,
    fill=data.gravity_bouguer_mgal,
    cmap=True,
    style="c4p",
    projection="M15c", 
    frame=True,
)
fig.colorbar(frame='af+l"Bouguer disturbance [mGal]"')
fig.show()

## Get residual

In [ ]:
# Create a set of deep sources at a depth of 500 km
deep_sources = hm.EquivalentSources(damping=1000, depth=500e3)

# Fit the sources to the data
deep_sources.fit((data.easting_m, data.northing_m, data.height_geometric_m), data.gravity_bouguer_mgal)

# Use the sources to predict the regional field
regional = deep_sources.predict((data.easting_m, data.northing_m, data.height_geometric_m))

# Calculate a residual field (which is what we want)
residual =  data.gravity_bouguer_mgal - regional

data = data.assign(gravity_residual_mgal=residual)
data

In [ ]:
fig = pygmt.Figure()
maxabs = vd.maxabs(data.gravity_residual_mgal)
pygmt.makecpt(cmap="polar+h", series=[-maxabs, maxabs])
fig.plot(
    x=data.longitude,
    y=data.latitude,
    fill=data.gravity_residual_mgal,
    cmap=True,
    style="c4p",
    projection="M15c", 
    frame=True,
)
fig.colorbar(frame='af+l"Gravity residual [mGal]"')
fig.show()

## Grid

In [ ]:
eqs = hm.EquivalentSources(damping=10, depth=10e3)
eqs.fit(coordinates, data.gravity_residual_mgal)

In [ ]:
grid_coords = vd.grid_coordinates(
    region=region,
    spacing=2/60, # decimal degrees
    extra_coords=2200, # height in meters
)
residual_grid = eqs.grid(
    coordinates=grid_coords,
    data_names=["gravity_residual"],
    dims=("latitude", "longitude"),
    projection=projection,
)
residual_grid

In [ ]:
fig = pygmt.Figure()
scale = vd.maxabs(residual_grid.gravity_residual)
pygmt.makecpt(cmap="polar", series=[-scale, scale], no_bg=True)
fig.grdimage(
    residual_grid.gravity_residual,
    shading="+a45+nt0.15",
    projection="M15c",
    frame=True,
)
fig.colorbar(frame='af+l"residual field [mGal]"')
fig.plot(
    x=data.longitude,
    y=data.latitude,
    style="c0.02c",
    color="black",
)
fig.show()